<a href="https://colab.research.google.com/github/Nanxi-lu/BSA_Big_data/blob/main/W04_postgis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 14.6 MB/s eta 0:00:00


# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [3]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.8/662.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 17.4 

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [4]:
import os
import zipfile
import duckdb
import pandas as pd
import leafmap

zip_url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
zip_path = "/content/nyc_data.zip"
extract_dir = "/content/nyc_data_extracted"

leafmap.download_file(zip_url, output=zip_path)
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_dir)

os.listdir(extract_dir)[:20]


Downloading...
From: https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 73.2MB/s]


Extracting files...


['nyc_homicides.prj',
 'nyc_streets.dbf',
 'nyc_homicides.shp',
 'nyc_streets.shp',
 'nyc_census_blocks.shx',
 'nyc_neighborhoods.shx',
 'README.txt',
 'nyc_census_blocks.dbf',
 'nyc_streets.shx',
 'nyc_subway_stations.shp',
 'nyc_neighborhoods.prj',
 'nyc_homicides.shx',
 'nyc_streets.prj',
 'nyc_subway_stations.shx',
 'nyc_census_blocks.shp',
 'nyc_subway_stations.dbf',
 'nyc_neighborhoods.shp',
 'nyc_subway_stations.prj',
 'nyc_homicides.dbf',
 'nyc_neighborhoods.dbf']

In [5]:
import duckdb

con = duckdb.connect("/content/nyc.duckdb")

con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
con.execute("""
CREATE OR REPLACE TABLE nyc_census_blocks AS
SELECT * FROM ST_Read('/content/nyc_data_extracted/nyc_census_blocks.shp');
""")

con.execute("""
CREATE OR REPLACE TABLE nyc_homicides AS
SELECT * FROM ST_Read('/content/nyc_data_extracted/nyc_homicides.shp');
""")

con.execute("""
CREATE OR REPLACE TABLE nyc_neighborhoods AS
SELECT * FROM ST_Read('/content/nyc_data_extracted/nyc_neighborhoods.shp');
""")

con.execute("""
CREATE OR REPLACE TABLE nyc_streets AS
SELECT * FROM ST_Read('/content/nyc_data_extracted/nyc_streets.shp');
""")

con.execute("""
CREATE OR REPLACE TABLE nyc_subway_stations AS
SELECT * FROM ST_Read('/content/nyc_data_extracted/nyc_subway_stations.shp');
""")


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [7]:


# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [8]:
try:
    con.close()
except NameError:
    pass


In [9]:
import duckdb
import pandas as pd

%reload_ext sql
%pip -q install duckdb-engine
%sql duckdb:////content/nyc.duckdb

In [10]:
%%sql
INSTALL spatial;
LOAD spatial;

SELECT
  s.name AS station,
  s.routes AS routes
FROM nyc_neighborhoods n
JOIN nyc_subway_stations s
  ON ST_Within(s.geom, n.geom)
WHERE n.name = 'Little Italy';


,station,routes
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [14]:
%%sql
INSTALL spatial;
LOAD spatial;

SELECT DISTINCT
  n.name AS neighborhood
FROM nyc_subway_stations s
JOIN nyc_neighborhoods n
  ON ST_Within(s.geom, n.geom)
WHERE strpos(s.routes, '6') > 0;


,neighborhood
0,Gramercy
1,Murray Hill
2,Soundview
3,Upper East Side
4,Financial District
5,Hunts Point
6,East Harlem
7,Little Italy
8,Yorkville
9,Chinatown


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [15]:
%%sql
INSTALL spatial;
LOAD spatial;

SELECT SUM(c.popn_total) AS evacuated_population
FROM nyc_neighborhoods n
JOIN nyc_census_blocks c
  ON ST_Intersects(n.geom, c.geom)
WHERE n.name = 'Battery Park';

,evacuated_population
0,17153.0


4. **What neighborhood has the highest population density (persons/km2)?**


In [17]:
%%sql
INSTALL spatial;
LOAD spatial;

SELECT
  n.name AS neighborhood,
  SUM(c.popn_total) / (ST_Area(n.geom) / 1000000.0) AS density_persons_per_km2
FROM nyc_neighborhoods n
JOIN nyc_census_blocks c
ON ST_Within(c.geom, n.geom)
GROUP BY n.name, n.geom
ORDER BY density_persons_per_km2 DESC LIMIT 2;


,neighborhood,density_persons_per_km2
0,North Sutton Area,47112.378626
1,Upper East Side,45502.141979


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

# Task
Import the shapefiles "nyc_census_blocks.shp", "nyc_homicides.shp", "nyc_neighborhoods.shp", "nyc_streets.shp", and "nyc_subway_stations.shp" from the `nyc_data_extracted` directory into a DuckDB database named `nyc.duckdb` as tables with the same names as the shapefiles, and then execute the SQL query to find subway stations and routes in 'Little Italy'.

## Connect to DuckDB Database

### Subtask:
Ensure a persistent connection to the `nyc.duckdb` database located in the `nyc_data_extracted` directory. This will be the database where all shapefiles are imported.


**Reasoning**:
The previous attempt to connect to the DuckDB database failed because the path was incorrect. I need to connect to the `nyc.duckdb` file located within the `nyc_data_extracted` directory using the correct relative path and the `%sql` magic command.



In [ ]:
%%sql
ATTACH './nyc_data_extracted/nyc.duckdb' AS nyc_data;
USE nyc_data;

,Success


**Reasoning**:
I need to ensure a persistent connection to the specified DuckDB database. The previous step attempted to attach and use the database, but the subtask explicitly asks for a direct connection using `%sql duckdb:///./nyc_data_extracted/nyc.duckdb`. This command will establish the primary connection for subsequent SQL operations.



In [ ]:
%sql duckdb:///./nyc_data_extracted/nyc.duckdb

**Reasoning**:
I need to ensure a persistent connection to the specified DuckDB database. The previous step attempted to attach and use the database, but the subtask explicitly asks for a direct connection using `%sql duckdb:///./nyc_data_extracted/nyc.duckdb`. This command will establish the primary connection for subsequent SQL operations.



In [ ]:
%sql duckdb:///./nyc_data_extracted/nyc.duckdb

## Load Spatial Extension

### Subtask:
Confirm that the DuckDB spatial extension is loaded, as it's required for reading spatial data like shapefiles.


**Reasoning**:
To confirm that the DuckDB spatial extension is loaded, I need to first install it and then load it, as specified in the instructions. This will ensure that spatial functions are available for subsequent operations.



In [ ]:
%%sql
INSTALL spatial;
LOAD spatial;

,Success


## Import nyc_census_blocks

### Subtask:
Import the nyc_census_blocks.shp file from the nyc_data_extracted directory into a new table named nyc_census_blocks in the DuckDB database using ST_READ.


**Reasoning**:
I need to import the `nyc_census_blocks.shp` shapefile into the DuckDB database as a new table named `nyc_census_blocks` using the `ST_READ` function as specified in the subtask.



In [ ]:
%%sql
CREATE OR REPLACE TABLE nyc_census_blocks AS
SELECT * FROM ST_READ('./nyc_data_extracted/nyc_census_blocks.shp');

,Success


**Reasoning**:
I need to import the `nyc_homicides.shp` shapefile into the DuckDB database as a new table named `nyc_homicides` using the `ST_READ` function as specified in the subtask.



In [ ]:
%%sql
CREATE OR REPLACE TABLE nyc_homicides AS
SELECT * FROM ST_READ('./nyc_data_extracted/nyc_homicides.shp');

,Success


**Reasoning**:
I need to import the `nyc_neighborhoods.shp` shapefile into the DuckDB database as a new table named `nyc_neighborhoods` using the `ST_READ` function.



In [ ]:
%%sql
CREATE OR REPLACE TABLE nyc_neighborhoods AS
SELECT * FROM ST_READ('./nyc_data_extracted/nyc_neighborhoods.shp');

,Success


**Reasoning**:
I need to import the `nyc_streets.shp` shapefile into the DuckDB database as a new table named `nyc_streets` using the `ST_READ` function.



In [ ]:
%%sql
CREATE OR REPLACE TABLE nyc_streets AS
SELECT * FROM ST_READ('./nyc_data_extracted/nyc_streets.shp');

,Success


**Reasoning**:
I need to import the `nyc_subway_stations.shp` shapefile into the DuckDB database as a new table named `nyc_subway_stations` using the `ST_READ` function.



In [ ]:
%%sql
CREATE OR REPLACE TABLE nyc_subway_stations AS
SELECT * FROM ST_READ('./nyc_data_extracted/nyc_subway_stations.shp');

,Success


**Reasoning**:
All the required shapefiles have been imported into the DuckDB database. Now I can execute the SQL query to find the subway stations and routes in 'Little Italy' as requested in the main task. This query directly addresses the first question posed in the notebook.



In [ ]:
%%sql
SELECT
  s.name  AS station,
  s.routes AS routes
FROM nyc_neighborhoods n
JOIN nyc_subway_stations s
  ON ST_Within(s.geom, n.geom)
WHERE n.name = 'Little Italy';

,station,routes
0,Spring St,6


## Final Task

### Subtask:
Confirm that all shapefiles have been imported correctly and the user's query has been successfully executed, providing the requested information.


## Summary:

### Q&A
The shapefiles `nyc_census_blocks.shp`, `nyc_homicides.shp`, `nyc_neighborhoods.shp`, `nyc_streets.shp`, and `nyc_subway_stations.shp` have all been imported correctly into the `nyc.duckdb` database as tables with the same names. The user's query to find subway stations and routes in 'Little Italy' was successfully executed. The query returned one subway station, "Spring St," which is served by route "6."

### Data Analysis Key Findings
*   A persistent connection to the `nyc.duckdb` database in the `nyc_data_extracted` directory was successfully established.
*   The DuckDB spatial extension was successfully installed and loaded, enabling spatial data operations.
*   All five required shapefiles were successfully imported into the DuckDB database, creating tables with corresponding names:
    *   `nyc_census_blocks` from `nyc_census_blocks.shp`
    *   `nyc_homicides` from `nyc_homicides.shp`
    *   `nyc_neighborhoods` from `nyc_neighborhoods.shp`
    *   `nyc_streets` from `nyc_streets.shp`
    *   `nyc_subway_stations` from `nyc_subway_stations.shp`
*   The SQL query to identify subway stations and routes within the 'Little Italy' neighborhood was successfully executed using a spatial join (`ST_Within`).
*   The query identified one subway station in 'Little Italy': "Spring St" served by route "6".

### Insights or Next Steps
*   The successful import and querying demonstrate the effective setup of a spatial database using DuckDB and its spatial extension, allowing for geographic analysis.
*   The established database and imported datasets are now ready for more complex spatial analyses or visualizations involving these New York City geographic layers.
